
<a href="https://colab.research.google.com/github/isikus/MachineScorning/blob/main/Нагенерировать%20машинного%20отупения%20на%20ruGPT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Нагенерировать машинного отупения на ruGPT3

In [1]:
import os
import json

def is_empty(fpath, default_lines=[["Дополни эту фразу", "И эту фразу тоже дополни"], ["Первый пример запроса", "Из нескольких строк", "", "И второй пример", "Я тоже придумать смог"]]):
  if not os.path.isfile(fpath):
    return True
  if not (os.path.getsize(fpath) > 0):
    return True
  if default_lines:
    with open(fpath, "r") as infile:
      lines = infile.read().split("\n")
    for deflines in default_lines:
      if lines == deflines:
        return True
  return False
  

def get_lbc(model_meta):
  lbc = model_meta["line_break_character"]
  lbc = lbc.replace("\s", " ")
  return lbc

#@markdown Для начала выберите модель:

Модель = "Нейроописания товаров"  #@param ["Нейроописания товаров", "Нейроинструкции на все случаи жизни", "Нейротексты «Короля и Шута»", "Нейроновости Северной Кореи", "Нейропоздравления с новым годом"]

desc_dict = {
    "Нейроописания товаров": "products_desc",
    "Нейроинструкции на все случаи жизни": "wikihowto",
    "Нейротексты «Короля и Шута»": "kishverses",
    "Нейроновости Северной Кореи": "rukcna",
    "Нейропоздравления с новым годом": "greetings"
}

model = desc_dict[Модель]

!wget -q --show-progress https://storage.googleapis.com/ml-bucket-isikus/machine-scorning/{model}.tar.gz
!mkdir {model}
!tar -xzf {model}.tar.gz -C {model}

try:
  with open(model+"/meta.json", "r") as injson:
    model_meta = json.load(injson)
  print('\x1b[1m'+Модель + ": модель подготовлена"+'\x1b[0m')
except Exception as e:
  print('\x1b[1;31m'+"Что-то сломалось – модель не готова :("+'\x1b[0m')

products_desc.tar.g 100%[===================>] 443.71M   279MB/s    in 1.6s    
Нейроописания товаров: модель подготовлена


Теперь запишите варианты начала сообщений, которые должна дополнить модель, сразу под текстом `%%writefile input.txt`, по 1 разному началу на строку:

In [2]:
%%writefile input.txt
Дополни эту фразу
И эту фразу тоже дополни

Writing input.txt


**Важно:** если ваши запросы содержат переносы строк (например, это начало стихотворения), вместо предыдущей клетки, заполните клетку ниже, разбивая ваш ввод двумя переносами строк. Код ниже заменит переносы строк на специальный символ – это необходимо для корректной работы модели.

In [3]:
%%writefile input_with_lines.txt
Первый пример запроса
Из нескольких строк

И второй пример
Я тоже придумать смог

Writing input_with_lines.txt


Если у вас совсем нет идей, запустите код ниже – он скачает примеры запросов по умолчанию для вашей модели:

In [ ]:
if is_empty("input.txt") and is_empty("input_with_lines.txt"):
  !cp {model}/example_input.txt input.txt
  print('\x1b[1;31m'+"Загружены примеры по умолчанию"+'\x1b[0m')

In [5]:
#@markdown Наконец, укажите, сколько вариантов выдачи нужно сгенерировать модели на каждый из запросов (от 1 до 30):

Повторов = 4  #@param {"type": "integer"}
takes = Повторов

if takes < 1:
  takes = 1
  print("Это маловато. Установили значение " + str(takes) + ".")
if takes > 30:
  takes = 30
  print("Это многовато. Установили значение " + str(takes) + ".")
if type(takes) != int:
  takes = 4
  print("Что-то пошло не так. Установили значение " + str(takes) + ".")

## Подготавливаем среду для модели
Сами файлы скачаны, но нужно ещё выполнить код, чтобы подготовить модель к работе.

Проверяем, что мы подключены к видеокарте: команда в клетке ниже должна работать и показывать показывать примерно 12000-15000 Мб свободной памяти:

In [6]:
!nvidia-smi

Sun Feb 14 12:59:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Устанавливаем нужные версии библиотек:

In [7]:
!pip3 install -q urllib3==1.25.4 transformers==2.8.0

print('\n'+'\x1b[1m'+"Требуемые версии пакетов установлены"+'\x1b[0m')

     |████████████████████████████████| 133kB 15.7MB/s 
     |████████████████████████████████| 573kB 31.9MB/s 
     |████████████████████████████████| 133kB 60.0MB/s 
     |████████████████████████████████| 3.7MB 59.6MB/s 
     |████████████████████████████████| 1.2MB 55.5MB/s 
     |████████████████████████████████| 890kB 58.2MB/s 
     |████████████████████████████████| 7.2MB 53.5MB/s 
     |████████████████████████████████| 71kB 11.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.

Требуемые версии пакетов установлены


Скачиваем файл с исходным кодом для работы модели:

In [8]:
!wget -q --show-progress https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

print('\n'+'\x1b[1m'+"Код запуска модели загружен"+'\x1b[0m')

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      

Код запуска модели загружен


Запускаем код, необходимый для преобразования файлов и отображения выдачи:

In [9]:
import re
import os
import sys
import json
import random
from datetime import datetime

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

with open(model+"/meta.json", "r") as injson:
  model_meta = json.load(injson)

# converting inputs

s = ""

if not is_empty("input_with_lines.txt"):
  with open("input_with_lines.txt", "r", encoding="utf-8") as intxt:
    prompts = [get_lbc(model_meta).join([sp for sp in p.split("\n") if sp]) for p in intxt.read().split("\n\n") if p]
else:
  with open("input.txt", "r", encoding="utf-8") as intxt:
    prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * takes

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

print('\x1b[1m'+"Среда подготовлена"+'\x1b[0m')

Среда подготовлена


## Генерируем отупение

Теперь всё, наконец, готово к тому, чтобы модель сгенерировала нам выдачу. То, насколько серьёзный и осмысленный текст будет пытаться выдавать модель, можно регулировать по шкале **`Бредовость`**: здесь есть 5 степеней – от самой нормальной до самой странной (я предупреждал). Кроме этого, флажок **`Повторяться`** определяет, как будет работать модель: если галочку оставить, то первый результат будет очень похож не те, что есть в канале, а последующие будут с ним совпадать. А если снять галочку, то выдача модели каждый раз будет новая. Удачи!

In [10]:
#@markdown ### Начать генерацию!

Бредовость = 75 #@param {type: "slider", min: 0, max: 100, step: 25}
stlv = Бредовость

stup_cases = {
    0: {"k": 5, "p": 0.95, "temp": 1.0},
    25: {"k": 5, "p": 0.93, "temp": 2.0},
    50: {"k": 6, "p": 0.91, "temp": 3.0},
    75: {"k": 6, "p": 0.9, "temp": 4.0},
    100: {"k": 7, "p": 0.88, "temp": 5.0}
}

Повторяться = True   # @param {"type": "boolean"}
add_randomness = not Повторяться

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
printmd("Ключ случайного значения: **"+str(rs)+"**")

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k={str(stup_cases[stlv]["k"])} \
    --p={str(stup_cases[stlv]["p"])} \
    --seed={str(rs)} \
    --temperature={str(stup_cases[stlv]["temp"])} \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

resps = [r for r in responses.split("Context >>> ruGPT:\n") if r]
sps = [sp for sp in s.split("\n") if sp]
for sp, resp in zip(sps, resps):
  pt = re.search(re.compile(repr(re.sub(r"%[a-z]+", "(.*)", model_meta["prompt"]))[1:-1]), sp).group(1)
  rs, _ = re.findall(re.compile(re.escape(sp) + r"\n?(.*?)(" + ssc + "|$)", flags=re.DOTALL), resp)[0]
  pt = "<br>".join("**" + p + "**" for p in re.split(lbc, pt) if p)
  printmd(pt)
  rs = re.sub(lbc, "<br>", rs).strip()
  rs = re.sub(r"([^\s])<br>([^\s])", r"\1 <br> \2", rs)
  printmd(rs[:rs.find("Context >>")] + "<br>")

Ключ случайного значения: **42**

**Тупая дичь**

2.7-2.8гб <br> Тумба, тумба под ТВ\3D <br> <br>Станки <br> МРЗ\МТЗ\ЛБП\ <br> МРЭИ(полировка) <br> <br>Педал\Педал <br> Микрон\Материнские <br> Порты\USB 2.1 и 2.3 <br> <br>DVD <br> CD-плееры <br> Win 7 64-бит\ <br> WiFi\HdD 802.11a <br> g<br> <br> Сетевые сети <br> Подписка на смс и Viber\Доступ в интернет <br> <br> <br> Потребительских привычек\Унитаз, Стиральные и посудомойка\Электроприборы\Электронный блок <br> Установки и системы видеонаблюдения\ <br> Удаленный доступ в интернет <br> <br> <br> Габаритов\ДxВxТ 58x35x8 <br> 10 мм <br> Вес <br> <br>2.8 Цена 2000<br>

**Тупая дичь**

Описание: Продам котиков-мальков с большими глазными впадинами и очень пушистошерстрым костяком (на фото видно), очень ласкова и активны, очень ждут своих заботливых хозяев, не кусаются (к еде неприхотливы). <br> <br>Приучен к лоток и корм для него. Приучена к когтеточкару и когтяточному когтеточкому.  Отдам бесплатно в хорошие заботные и заботливые ручки.<br> Цена 1000<br>

...

Плюшевый комбензон с капюшоном, утепление и защита от дождя\ <br> Описание <br> <br>Покупал за 45 т., но так и лежит в ящике((( Состояние нового, не подошёл размер, на мой рост 170 см. Смотреть в м-не Академгородок, могу подвезти до пл. Победы (м-он ул. Чадаевская) или до ост м-она Юбилейной. В подарок чехол для коляской (с капюшоном) на ноги. <br> <br> <br> Смотр в любое время! <br> <br>Звоните с 8 до 22 часов. Цена 2000 руб <br> <br>Забирать ул.Советская,14, ост. "Красноармейский рынок" или на ул.Красноармейская. Цена 2000₽ <end >><br<br>

**Разноброчл**

-вибро-коляску для малыша <br> Описание: Разнообразная развивалка от рождения до 3 лет, все игрушки в идеальном состоянии! <br> <br> <br> - 3 положения <br> <br>- съемные мягкие колёса с амортизацией <br> <br>- 5- точечный ремень для безопасности и безопасности <br> <br>- регулируемая подножка <br> <br> - 3 режима работы (на батарейках <br> <br> <br> В отличном состоянии! <br> <br>Пролежал без дела, продаем. <br> <br> <br> Возможен не адекватный обмен. <br> <br> <br> Звонить по указанному номеру. Цена 1000<br>

**Разноброчл**

-кресла <br> детские качели <br> Описание: Коляски б\у <br> к в идеальном состоянии <br> <br> <br> Просторное, удобное и мягкое место, на функциональных и мягких ножек, легко складывается для компактного размещения <br> <br>В комплект входит съемная подставка для ножек (на нее легко можно посадить малыша, например) <br> <br>Просторные колеса обеспечивают мягкий тормоз в любом положении, благодаря съемным боковыми колесиям <br> <br>В сложенном положении занимает мало места в машине или путешествии. <br> <br>Складывающийся механизм регулировки высоты сидении и регулировка спинок обеспечивают ребенку комфорт при длительных прогулкам в помещении <br> <br>Рама выполнена в форме трайбла <br> <br>Вместительный багажник с возможностью доступа в багажничок с кнопки на задней части сиденья, а также имеется дополнительный козырный карман для мелких предметов (кресла). <br> <br> <br> Крепится на рама с помощью двух пластиковых элементов: колеса с фиксаторами <br> насосами (для детей весом не более 10-18 килограмм) и съемные задние колесницы. В комплект входит съемная корзина для игрушек. <br> <br>Пятиточечные надежные страховочные ремни <br> <br>Вместимость для ребенка с 1 года: 3 позиции наклона, 6 уровней спинки; съемная перекладина для защиты головы ребенка <br> ребёнка от случайного сложения; 5 уровней высоты сиденья <br> кожи. <br> <br> <br> В подарок к комплекту идёт: <br> <br>1. Кресла-кроватки для взрослых (2 шт). Цена 2000₽ <br> <br>

## Больше отупения!

Хотите попробовать другие фразы? Тогда впишите их в клетку под этим текстом, как до этого, и запустите код ниже. Не забудьте поменять `%%writefile input.txt` на `%%writefile input_with_lines.txt`, если ваши запросы содержат переносы строк!

In [ ]:
%%writefile input.txt


Overwriting input.txt


In [11]:
#@markdown ### Генерируем новую выдачу

Повторов = 3  #@param {"type": "integer"}
takes = Повторов

if takes < 1:
  takes = 1
  print("Маловато повторов. Установлено значение " + str(takes) + ".")
if takes > 30:
  takes = 30
  print("Маловато повторов. Установлено значение " + str(takes) + ".")
if type(takes) != int:
  takes = 4
  print("Что-то пошло не так с количеством повторов. Установлено значение " + str(takes) + ".")

s = ""

if not is_empty("input_with_lines.txt"):
  with open("input_with_lines.txt", "r", encoding="utf-8") as intxt:
    prompts = [get_lbc(model_meta).join([sp for sp in p.split("\n") if sp]) for p in intxt.read().split("\n\n") if p]
else:
  with open("input.txt", "r", encoding="utf-8") as intxt:
    prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * takes

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

Бредовость = 25 #@param {type: "slider", min: 0, max: 100, step: 25}
stlv = Бредовость

Повторяться = False   # @param {"type": "boolean"}
add_randomness = not Повторяться

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
printmd("Ключ случайного значения: **"+str(rs)+"**")

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k={str(stup_cases[stlv]["k"])} \
    --p={str(stup_cases[stlv]["p"])} \
    --seed={str(rs)} \
    --temperature={str(stup_cases[stlv]["temp"])} \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

resps = [r for r in responses.split("Context >>> ruGPT:\n") if r]
sps = [sp for sp in s.split("\n") if sp]
for sp, resp in zip(sps, resps):
  pt = re.search(re.compile(repr(re.sub(r"%[a-z]+", "(.*)", model_meta["prompt"]))[1:-1]), sp).group(1)
  rs, _ = re.findall(re.compile(re.escape(sp) + r"\n?(.*?)(" + ssc + "|$)", flags=re.DOTALL), resp)[0]
  pt = "<br>".join("**" + p + "**" for p in re.split(lbc, pt) if p)
  printmd(pt)
  rs = re.sub(lbc, "<br>", rs).strip()
  rs = re.sub(r"([^\s])<br>([^\s])", r"\1 <br> \2", rs)
  printmd(rs[:rs.find("Context >>")] + "<br>")

Ключ случайного значения: **923**

**Тупая дичь**

1 <br> Описание: Продаю молодую, красивую, но в очень хорошем состоянии  дичку. Очень красивая, крупная. Родилась 21 января.  Клетка с окошком для купания. Цена 1500<br>

**Тупая дичь**

Описание: Продаётся тупая,клетчатая кошечка,девочка. Возраст 3 года. Очень ласковая и добрая,к еде не привередлива,кушает самостоятельно. Цена 100<br>

...

Описание: Продам детскую коляску 2в1 (2 блока) в отличном состоянии. В комплекте: люлька, прогулочный модуль, автолюлька, автозеркала, сумка для мамы, сумка для мамы, корзина для покупок. Цена 6000<br>

**Разноброчл**

Описание: Продам детский спортивный комплекс. Надувная подставка. В комплекте все что нужно для занятий. <br> <br> <br> Состоит из: <br> <br>1) надувной пластиковый каркас; <br> <br>2) каркас, на котором крепится подставка, с пластиковым подголовником и съемным козырьком; <br> <br>3) надувной матрас; <br> <br>5) кольца; Цена 2000<br>

**Разноброчл**

Пластиковый стаканчик (с крышкой) <br> Описание: Разборный пластиковый стаканчик ( с крышкой). Цвет - белый с черным. Цвет бутылки - белый с черным. Размер: высота - 2.7 м, ширина  1.7 м <br> <br>В отличном состоянии, без дефектов. Цена 500₽ <br> <br>

На этом всё. Получайте удовольствие (надеюсь)!